In [1]:
import sys
sys.path.append("../../")
import fibi.src.pie_chart as pie_chart, fibi.src.statistical_test as statistical_test
import pathlib as p
import numpy as np
import tqdm
import pandas as pd
import pyarrow.parquet as pq
root = p.Path("../data/algorithms_out/clustering/data_Clustering_Normaly distr. points_quadNorm.parquet").resolve()
root_folder = p.Path("../").resolve()
table = pq.read_table(root)
df = table.to_pandas()
df.head()

,num_iter,num_iter_glob,init_cost,final_cost,duration,SEED_GLOB,SEED_PROBLEM,SEED_ASSIGN,DATASET,FI_BI,IMPR,NUM_POINTS,NUM_DIM,NUM_CLUST,filename,KEY,PROBLEM,init_meth,ratio
0,21.0,46.0,133602.70345,95312.38142,0.000057,2.0,0.0,0.0,Normaly distr. points,BI,RAND,40,2.0,2,dataset3,2.0,Clustering,random,0.71340
1,27.0,92.0,148363.32162,91554.89098,0.000094,3.0,0.0,0.0,Normaly distr. points,BI,RAND,50,2.0,2,dataset3,3.0,Clustering,random,0.61710
2,32.0,96.0,134612.81367,78962.48410,0.000129,4.0,0.0,0.0,Normaly distr. points,BI,RAND,60,2.0,2,dataset3,4.0,Clustering,random,0.58659
3,30.0,124.0,352882.52743,180073.89063,0.000112,5.0,0.0,0.0,Normaly distr. points,BI,RAND,70,2.0,2,dataset3,5.0,Clustering,random,0.51029
4,53.0,228.0,614488.58345,325035.18487,0.000418,10.0,0.0,0.0,Normaly distr. points,BI,RAND,120,2.0,2,dataset3,10.0,Clustering,random,0.52895


In [2]:
list_cases_per_init = {}
for initialization, df_init in df.groupby("IMPR"):
    L_cases = []
    for instance, df_instance in df_init.groupby(["NUM_POINTS", "NUM_CLUST"]):
        df_instance.sort_values(by="SEED_GLOB", inplace=True)
        bi = df_instance.query("FI_BI == 'BI'")["final_cost"].to_numpy()
        fi = df_instance.query("FI_BI == 'FI'")["final_cost"].to_numpy()
        init = df_instance.query("FI_BI == 'FI'")["init_cost"].to_numpy()
        diff = (bi-fi)/init
        case = pie_chart.get_case_from_diff(
            diff=diff,
            maximization=False,
            init_random=initialization == "RAND"
        )
        L_cases.append(case)

    assert len(L_cases) > 0, "More than 0 samples needed"
    # We build the input for the pie chart function (one list asssociated to the initialization random)
    list_cases_per_init[initialization] = L_cases


problem = "mssc"
dataset = "randomNormal"
# We create the piechart
figure_text = pie_chart.make_latex_piecharts_figure_for_datasets_of_problem(
    list_cases_per_init=list_cases_per_init,
    problem=problem,
    dataset=dataset,
    template_folder=root_folder / "data" / "templates" / "latex"
)
# And save it
with open(root_folder / "data" / "out_diagrams" / f"{problem}_{dataset}.tex", "w", encoding="utf-8") as fp:
    fp.write(figure_text)